# 8. Graphical Models

In [1]:
from prml import bayesnet as bn

In [2]:
b = bn.discrete([0.1, 0.9])
f = bn.discrete([0.1, 0.9])

g = bn.discrete([[[0.9, 0.8], [0.8, 0.2]], [[0.1, 0.2], [0.2, 0.8]]], b, f)

In [3]:
print("b:", b)
print("f:", f)
print("g:", g)

b: DiscreteVariable(proba=[0.1 0.9])
f: DiscreteVariable(proba=[0.1 0.9])
g: DiscreteVariable(proba=[0.315 0.685])


In [4]:
g.observe(0)

In [5]:
print("b:", b)
print("f:", f)
print("g:", g)

b: DiscreteVariable(prior=[0.1 0.9], posterior=[0.25714286 0.74285714])
f: DiscreteVariable(prior=[0.1 0.9], posterior=[0.25714286 0.74285714])
g: DiscreteVariable(observed=[1. 0.])


In [6]:
b.observe(0)

In [7]:
print("b:", b)
print("f:", f)
print("g:", g)

b: DiscreteVariable(observed=[1. 0.])
f: DiscreteVariable(prior=[0.1 0.9], posterior=[0.11111111 0.88888889])
g: DiscreteVariable(observed=[1. 0.])
